In [1]:
import yaml
import sys
import os
import numpy as np
import torch 
print('START')
os.chdir('/home/jonfrey/PLR3')
sys.path.insert(0, os.getcwd())
sys.path.append(os.path.join(os.getcwd() + '/src'))
sys.path.append(os.path.join(os.getcwd() + '/lib'))

import loaders_v2
from loaders_v2 import GenericDataset
from rotations import * 

exp_cfg_path = '/home/jonfrey/PLR3/yaml/exp/exp_ws_deepim_debug_natrix.yml'
env_cfg_path = '/home/jonfrey/PLR3/yaml/env/env_natrix_jonas.yml'

def load_from_file(p):
    if os.path.isfile(p):
        with open(p, 'r') as f:
            data = yaml.safe_load(f)
    else:
        raise ValueError
    return data

exp = load_from_file(exp_cfg_path)
env = load_from_file(env_cfg_path)

dataset_train = GenericDataset(
    cfg_d=exp['d_train'],
    cfg_env=env)

START
Finished loading meshes 0.04542970657348633


In [2]:
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw

class Drawer():
    def __init__(self):
        self.im_in_plot = 0
        self.data = []
    def disp_img_1d(self,img,hold=False):
        self.data.append(img)
        
        if not hold:
            fig = plt.figure(figsize=(6*2*len(self.data),7))
            ax = []
            for j,a in enumerate(self.data):
                ax.append( fig.add_subplot(1,len(self.data), j+1)  )
                
                ax[-1].get_xaxis().set_visible(False)
                ax[-1].get_yaxis().set_visible(False)
                pos = ax[-1].imshow( a, cmap='Blues' )
                fig.colorbar(pos, ax=ax[-1])
            plt.show()
            self.data = []
            self.ax = []
            print('SHOW')
draw = Drawer()

In [3]:
import time
st = time.time()
for j in range(0,1):
    i = np.random.randint(0,10000)
    batch = dataset_train[i][0] #bann 10450
    points, choose, img, target, model_points, idx = batch[0:6]
    depth_img, label_img, img_orig, cam = batch[6:10]
    gt_rot_wxyz, gt_trans, unique_desig = batch[10:13]

    real_img, render_img, real_d, render_d, gt_label_cropped = batch[13:18]
    pred_rot_wxyz, pred_trans, pred_points, h_render,h_real, render_img_original = batch[18:24]
    u_map, v_map, flow_mask, bb = batch[24:]
    real_tl, real_br, ren_tl, ren_br = bb
#     from visu import Visualizer
#     visualizer = Visualizer('/home/jonfrey/Debug')
#     visualizer.plot_corrospondence('tag', 2, u_map, v_map, flow_mask, real_img, render_img, store=False, jupyter=True)

#     draw = Drawer()
#     draw.disp_img_1d( u_map, True)
#     draw.disp_img_1d( v_map, True)
#     draw.disp_img_1d( flow_mask )
#     # draw.disp_img_1d( gt_label_cropped )

# #     c1[ int(b_real.tl[0]):int(b_real.br[0]), int(b_real.tl[1]):int(b_real.tl[1]+5) ] = 100000
# #     c1[ int(b_real.tl[0]):int(b_real.br[0]), int(b_real.br[1]):int(b_real.br[1]+5) ] = 100000
#     img_orig [ int(b_real.tl[0]):int(b_real.br[0]), int(b_real.tl[1]):int(b_real.tl[1]+5),:] = torch.tensor( [0,255,0] )
#     img_orig [ int(b_real.tl[0]):int(b_real.br[0]), int(b_real.br[1]):int(b_real.br[1]+5),:] = torch.tensor( [0,255,0] )
#     print(b_ren.br, b_ren.tl)
    
#     render_img_original [ int(b_ren.tl[0]):int(b_ren.br[0]), int(b_ren.tl[1]):int(b_ren.tl[1]+5),:] = torch.tensor( [0,255,0] )
#     render_img_original [ int(b_ren.tl[0]):int(b_ren.br[0]), int(b_ren.br[1]):int(b_ren.br[1]+5),:] = torch.tensor( [0,255,0] )

#     display( Image.fromarray(np.uint8(img_orig.numpy())) )
#     display( Image.fromarray(np.uint8(render_img_original.numpy())) )
print(time.time()-st)
bb



/home/jonfrey/PLR3/src/helper/bounding_box.py:269: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629427478/work/torch/csrc/utils/python_arg_parser.cpp:766.)
  masked_idx = (d != 0).nonzero()


0.8038341999053955


(tensor([114., -30.], dtype=torch.float64),
 tensor([471., 446.], dtype=torch.float64),
 tensor([27., 13.]),
 tensor([493., 633.]))

In [37]:
# What we gonna do: 
# 1. Given the Depth Map of the Real Image calculate the PCD machting disparity pixels. 
# 2. Given the Depth Map of the Render Image calculate the PCD for the rendered Image. 

# Visualize the vectors in the "Current Object Center Frame" for rendered and real 
from visu import plot_pcd


# real_d[0][flow_mask]
a = float((real_br[0]-real_tl[0])/480)
b = float((real_br[1]-real_tl[1])/640)
grid_real_h, grid_real_w = np.mgrid[int(real_tl[0]) :int(real_br[0]):a , int(real_tl[1]) :int(real_br[1]):b]

a = float((ren_br[0]-ren_tl[0])/480)
b = float((ren_br[1]-ren_tl[1])/640)
grid_ren_h, grid_ren_w = np.mgrid[int(ren_tl[0]) :int(ren_br[0]):a , int(ren_tl[1]) :int(ren_br[1]):b]


print(grid_real_w[flow_mask].shape, grid_real_h[flow_mask], np.ones(grid_real_h.shape)[flow_mask])

cam_scale = 10000

real_pixels = np.stack( [grid_real_w[flow_mask], grid_real_h[flow_mask], np.ones(grid_real_h.shape)[flow_mask]], axis=1 )
K = np.array( [[int(cam[2]),0,int(cam[0])],[0,int(cam[3]),int(cam[1])],[0,0,1]] )
K_inv = np.linalg.inv(K)

P_real = K_inv @ real_pixels.T 
P_real = P_real * real_d[0][flow_mask].numpy() / cam_scale


K_ren = dataset_train._backend.get_camera('data_syn/000001', K=True)
K_ren_inv = np.linalg.inv(K_ren)
ren_pixels = np.stack( [grid_ren_w[flow_mask]+v_map[flow_mask].numpy(), 
                        grid_ren_h[flow_mask]+u_map[flow_mask].numpy(),
                        np.ones(grid_ren_h.shape)[flow_mask]], 
                        axis=1 )

P_ren = K_ren_inv @ ren_pixels.T 
P_ren = P_ren * render_d[0][flow_mask].numpy() / cam_scale

P_real = P_real.T
P_ren = P_ren.T
plot_pcd(P_real[::100,:], point_size=0.005, c='g')
plot_pcd(P_ren[::100,:], point_size=0.005, c='g')



P_real_in_center = P_real - h_real[:3,3].numpy()
P_ren_in_center = P_ren - h_render[:3,3].numpy()


plot_pcd(P_real_in_center[::100,:], point_size=0.005, c='r')
plot_pcd(P_ren_in_center[::100,:], point_size=0.005, c='r')


(77539,) [136.3125 136.3125 136.3125 ... 447.2    447.2    447.2   ] [1. 1. 1. ... 1. 1. 1.]


Output()

Output()

Output()

Output()

In [36]:
P_real

array([[-0.10870237, -0.0889602 ,  0.90670361],
       [-0.10806975, -0.08896019,  0.90670352],
       [-0.10743715, -0.0889602 ,  0.90670361],
       ...,
       [-0.06281516,  0.17085785,  0.88411895],
       [-0.0622506 ,  0.17100151,  0.8848623 ],
       [-0.061685  ,  0.17114514,  0.88560557]])

In [24]:
# real_d[0][flow_mask]
torch.max( render_d[0][flow_mask] )
torch.max( real_d[0][flow_mask] )

tensor(9930.)